In [ ]:
!pip install requests
!pip install json
!pip install csv

In [1]:
import requests
import json
import csv

In [2]:
# URL containing the JSON data
json_url = "https://mesonet.agron.iastate.edu/json/nwstext_search.py?sts=2008-01-01T00:00Z&ets=2023-12-31T00:00Z&awipsid=AFDAPX"

filter_words = ["mcs", "qcls", "mesoscale convective system", "quasi-linear convective system"]

unique_dates = set()

# Make an HTTP request to the URL
response = requests.get(json_url)

csv_data = []

if response.status_code == 200:
    # Parse the JSON data
    data = response.json()

    # Extract the name of the office
    office_name = data["results"][0]["cccc"] if "results" in data and data["results"] else "UnknownOffice"

    # Accessing information
    for result in data["results"]:
        # Check if the "data" field includes specific words (case-insensitive)
        if any(word.lower() in result["data"].lower() for word in filter_words):
            # Extract the unique date (year, month, and day)
            utc_valid_date = result["utcvalid"].split("T")[0]
            if utc_valid_date not in unique_dates:
                # Add the result to the CSV data list
                csv_data.append({"UTC Valid Date": utc_valid_date})
                unique_dates.add(utc_valid_date)

    # Export to CSV file with the office name
    csv_file_path = f"{office_name}_METR_4990_Dates.csv"
    with open(csv_file_path, "w", newline="", encoding="utf-8") as csvfile:
        fieldnames = ["UTC Valid Date"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header
        writer.writeheader()

        # Write the data
        for row in csv_data:
            writer.writerow(row)

        print(f"Results exported to {csv_file_path}")
else:
    print("Failed to retrieve data. Status code:", response.status_code)

Results exported to KAPX_METR_4990_Dates.csv
